# Load Library

In [ ]:
  !pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 5.3 MB/s eta 0:00:00


In [ ]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
pip install fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.8 MB/s eta 0:00:00


In [ ]:
pip install indonlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#import emoji
from sklearn.utils import resample
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from sklearn.utils.class_weight import compute_class_weight



# Read Dataset

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

# Cleaning Dataset

In [ ]:
# Function to clean data
def clean_tweet(tweet):
    import re

    # Define regex pattern to match usernames, hyperlinks, and old style retweets
    pattern = r'(@[\w/+=]+)|(https?://\S+)|(\[RE\s+\w+\])'

    # Remove usernames, hyperlinks, and old style retweets
    tweet = re.sub(pattern, '', tweet)

    # Remove numbers
    tweet = re.sub('[0-9]+', '', tweet)

    # Remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # Menghapus teks retweet gaya lama "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # Remove special characters using regular expressions (re) hashtag
    tweet = re.sub(r'[^\w\s]', '', tweet)

    return tweet

# Apply the cleaning function to every row in the 'text' column of the DataFrame
df['text'] = df['text'].apply(clean_tweet)


In [ ]:
#import stopword
from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';(', ':\U0001F92D'
    ])

# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
# Remove emoji
def remove_emojis(tweet):
    """
    Removes emojis from text while preserving punctuation, special characters,
    and characters from various languages (Korean, Chinese, Russian, German, Spanish, etc.).

    Args:
        tweet (str): The text to remove emojis from.

    Returns:
        str: The text with emojis removed.
    """

    # Unicode emoji character ranges (excluding flags)
    emoji_ranges = [
        u"\U0001F600-\U0001F64F",  # emoticons
        u"\U0001F300-\U0001F5FF",  # symbols & pictographs
        u"\U0001F680-\U0001F6FF",  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"   # miscellaneous symbols
    ]

    # Combine emoji ranges into a single pattern
    emoji_pattern = "|".join(emoji_ranges)

    # Create a pattern that matches any character except letters, numbers, punctuation,
    # whitespace, and characters from various languages (excluding flags)
    allowed_chars = r"[a-zA-Z0-9\s\-.,!?:;\(\)\[\]\{\}]"
    non_emoji_pattern = f"{allowed_chars}"

    # Combine the emoji and non-emoji patterns into a single pattern
    combined_pattern = f"(^{non_emoji_pattern})|({emoji_pattern})"

    # Remove emojis while keeping other characters
    return re.sub(combined_pattern, r"\1", tweet, flags=re.UNICODE)

df['text'] = df['text'].apply(remove_emojis)

In [ ]:
#Remove incorrect emoji
def incorrect_emoji(tweet):
    # Define patterns for incorrect encodings and non-standard characters
    encoding_pattern = re.compile(r"Ã[A-Za-z0-9ÂŒÂˆÂŠÂœ]+")

    # Remove these patterns from the text
    tweet = re.sub(encoding_pattern, '', tweet)

    # Remove any remaining non-standard characters
    tweet = re.sub(r'[^\x00-\x7F]+', '', tweet)

    return tweet

df['text'] = df['text'].apply(incorrect_emoji)

In [ ]:
#remove duplicate with same label
df = df.drop_duplicates(subset=['text', 'label'], keep='first')

In [ ]:
#remove duplicate with different label
df = df.drop_duplicates(subset=['text'], keep= False)

In [ ]:
from indoNLP.preprocessing import replace_slang

# Apply replace_slang to the 'text' column
df['text'] = df['text'].apply(lambda x: replace_slang(str(x)))

# Display the first few rows of the DataFrame to verify the changes
df.head()


,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,Anies dapat tepuk tangan meriah saat jadi Rekt...,Politik
2,memang benar sih pendukung ada yang goblok b...,Demografi
3,Sewaktu anies bersikap kritis ke kinerja pak p...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik


In [ ]:
from indoNLP.preprocessing import replace_word_elongation

# Apply replace_word_elongation function to the 'text' column of your DataFrame
df['text'] = df['text'].apply(replace_word_elongation)


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Download stopwords for the Indonesian language
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize the text
df['text'] = df['text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase

# Remove stopwords from tokens
stop_words = set(stopwords.words('indonesian'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

# Function to remove stopwords from list of words
def remove_stopwords(words):
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    cleaned_words = stopword_remover.remove(' '.join(words)).split()
    return cleaned_words

# Apply the remove_stopwords function to each list of words in the 'text' column
df['text'] = df['text'].apply(remove_stopwords)

# Join the list of words back into a string
df['text'] = df['text'].apply(' '.join)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Tokenize the text
df['text'] = df['text'].apply(lambda x: word_tokenize(str(x).lower()))  # Convert to lowercase

# Remove stopwords from tokens
stop_words = set(stopwords.words('indonesian'))
df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])


In [ ]:
#find the most use word

# Tokenize the text and count word frequencies
all_words = ' '.join([' '.join(text) for text in df['text']]).lower()
tokens = word_tokenize(all_words)
word_freq = Counter(tokens)

# Display the most common words
most_common_words = word_freq.most_common(10)  # Change 10 to the desired number of most common words
print("Most common words:")
for word, frequency in most_common_words:
    print(word, frequency)

Most common words:
anies 1880
ganjar 1760
prabowo 1561
pranowo 849
mahfud 813
indonesia 622
jnk 598
ganjarmahfud 596
ganjarpranowopilihanumat 558
capres 543


In [ ]:
#Remove kata-kata yang belum terdeteksi

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# List of words to remove
words_to_remove = [
    'ganjarmahfudrebound', 'md', 'baswedan', 'gibran', 'prabowogibran', 'orang',
    'subianto', 'amin', 'ga', 'no', 'amp', 'gm', 'nya', 'imin', 'aniesmuhaimin', 'prof', 'utk', 'lbihbaik',
    'beliau', 'abah', 'mas', 'pak', 'aja', 'kalo', 'presiden', 'gak', 'lu', 'gw', 'gue', 'lu', 'sm', 'w', 'doang', 'mah', 'ku', 'goblok',
    'retweet', 'reply', 'sih', 'dg', 'sj', 'jg'
]

# Add words from most_common_words to words_to_remove
for word, _ in most_common_words:
    words_to_remove.append(word)

def remove_specific_words(text):
    cleaned_words = []
    for word in text:
        # Check if the word is in the list of words to remove
        if word.lower() in words_to_remove:
            continue
        cleaned_words.append(word)
    return cleaned_words


In [ ]:
# Remove specific words from tokens
df['text'] = df['text'].apply(remove_specific_words)

# Join the list of words back into a string (if needed)
df['text'] = df['text'].apply(lambda x: ' '.join(x))


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['label'])

# Modeling

In [ ]:
# Split the data into training and testing sets
train_dt, test_dt = train_test_split(df, test_size=0.8, stratify=df['label'], random_state=42)

# Balance the training data if it's imbalanced
labels = train_dt['label'].value_counts()
max_count = labels.max()

balanced_train_dts = []

for label in labels.index:
    label_dt = train_dt[train_dt['label'] == label]
    if len(label_dt) < max_count:
        label_dt = resample(label_dt, replace=True, n_samples=max_count, random_state=42)
    balanced_train_dts.append(label_dt)

balanced_train_dt = pd.concat(balanced_train_dts)

# Define labels
labels_list = ["Ideologi", "Politik", "Ekonomi", "Sosial Budaya", "Pertahanan dan Keamanan", "Sumber Daya Alam", "Geografi", "Demografi"]

# Extract text data as a list of strings
train_texts = balanced_train_dt['text'].astype(str).tolist()
test_texts = test_dt['text'].astype(str).tolist()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch.optim as optim

# Load IndoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

In [ ]:
# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Convert labels to tensors
train_labels = torch.tensor(balanced_train_dt['encoded_label'].tolist())
test_labels = torch.tensor(test_dt['encoded_label'].tolist())

# Create TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [ ]:
# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
# optimizer = AdamW(model.parameters(), lr=2e-5)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

criterion = torch.nn.CrossEntropyLoss()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from transformers import BertForSequenceClassification, AdamW
import torch.optim as optim

# Load model architecture (already done)
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(labels))

# Define optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define loss function (CrossEntropyLoss for classification)
criterion = torch.nn.CrossEntropyLoss()

# Define device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
epochs = 10  # Example, adjust as needed
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_count = 0
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Calculate accuracy
            _, predicted = torch.max(logits, dim=1)
            total_correct += (predicted == labels).sum().item()
            total_count += labels.size(0)

        accuracy = balanced_accuracy_score(all_labels, all_predictions)
        print(f'Epoch {epoch + 1}/{epochs}, Validation Accuracy: {accuracy:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Validation Accuracy: 0.6508
Epoch 2/10, Validation Accuracy: 0.7300
Epoch 3/10, Validation Accuracy: 0.7258
Epoch 4/10, Validation Accuracy: 0.7235
Epoch 5/10, Validation Accuracy: 0.7374
Epoch 6/10, Validation Accuracy: 0.7400
Epoch 7/10, Validation Accuracy: 0.7380
Epoch 8/10, Validation Accuracy: 0.7386
Epoch 9/10, Validation Accuracy: 0.7386
Epoch 10/10, Validation Accuracy: 0.7394


In [ ]:
# Save the model at epoch 9
epoch_to_save = 9
model_dir = './model_checkpoint_epoch_9'
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

('./model_checkpoint_epoch_9/tokenizer_config.json',
 './model_checkpoint_epoch_9/special_tokens_map.json',
 './model_checkpoint_epoch_9/vocab.txt',
 './model_checkpoint_epoch_9/added_tokens.json')

In [ ]:
from sklearn.metrics import f1_score
#Validation
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Store predictions and true labels
        predictions.extend(np.argmax(logits, axis=1).flatten())
        true_labels.extend(label_ids.flatten())

# Calculate F1 score
f1 = f1_score(true_labels, predictions, average='weighted')
print(f'F1 Score: {f1:.4f}')

F1 Score: 0.7026
